<p><img src="https://files.cdn.thinkific.com/file_uploads/133292/images/a9e/c1a/bc3/P02.jpg"></p>

`MCSDSC02P01V3______`

`MCSDSC02P01V2______`

`MCSDSELIMC02P01V1010621`

## Постановка задачи

Необходимо изучить данные сервиса с тестового запуска на территории дизайн-завода Флакон и Хлебозавода №9 и ответить на вопросы, поставленные менеджером проекта:

На какие кафе приходится больше всего заказов? А на какие дни?
Чаще ли приложением пользуются именно в обеденное время?
Насколько большими компаниями  (или наоборот поодиночке) приходят в кафе?

## Импорт данных и библиотек

In [1]:
import pandas as pd


Прочитаем файл `bez_ocheredi.csv` и сохраним его в переменную `bo_data`.

In [2]:
#bo_data = pd.read_csv("https://github.com/Dobee1Knobee/sgsgsg/blob/d5e673aaa01436370f9a72d863aa4fc1ba6c7fcb/bez_ocheredi.csv")

bo_data = pd.read_csv('https://raw.githubusercontent.com/Dobee1Knobee/sgsgsg/d5e673aaa01436370f9a72d863aa4fc1ba6c7fcb/bez_ocheredi.csv')

Получим первые 10 строк таблицы.

In [3]:
bo_data.head(10)

,userID,ресторан,weekday,ordered,Столик для
0,b912c3c6a6ad,Iskra Bar,Monday,10:00,1
1,2dac0150717d,Iskra Bar,Monday,13:30,2
2,ba83aefab5dc,Iskra Bar,Tuesday,12:30,3
3,02996f1dd2ea,Iskra Bar,Wednesday,12:00,2
4,7c90be56d2ab,Iskra Bar,Thursday,14:30,3
5,51e0962387f7,Там Ем,Friday,12:30,2
6,c4f260a2b48a,NaN,Saturday,15:00,6
7,6b71309d6a8a,Крафтейнер,Monday,10:30,1
8,c5b969cb63a2,Бар Сибирь,Wednesday,15:30,2
9,b1570962e68c,Бар Сибирь,Wednesday,12:30,3


Получим общую информацию о данных из таблицы.

In [4]:
bo_data.info

<bound method DataFrame.info of            userID    ресторан     weekday  ordered    Столик для 
0     b912c3c6a6ad   Iskra Bar     Monday     10:00             1
1     2dac0150717d   Iskra Bar     Monday     13:30             2
2     ba83aefab5dc   Iskra Bar    Tuesday     12:30             3
3     02996f1dd2ea   Iskra Bar  Wednesday     12:00             2
4     7c90be56d2ab   Iskra Bar   Thursday     14:30             3
...            ...         ...        ...       ...           ...
1019  4b136de2c16a   Iskra Bar   Saturday     11:30             2
1020  9e46b91cc547  Бар Сибирь   Thursday     10:00             2
1021  5ca79042c44d      Там Ем   Thursday     14:00             3
1022  e0f9dfd1726c      Там Ем    Tuesday     17:30             1
1023  401d5fa8e55d      Там Ем  Wednesday     20:30             1

[1024 rows x 5 columns]>

Столбцы в `bo_data` содержат следующую информацию:

* user ID
* название ресторана
* название дня недели
* время заказа
* количество посетителей 


### Выводы

У нас есть полная база данных, состоящая из 1023 заказов, полученная от заведений на территории Флакона и Хлебзавода

## Предобработка данных

Получим названия столбцов.

In [5]:
bo_data.columns


Index(['userID ', ' ресторан ', ' weekday', ' ordered ', ' Столик для '], dtype='object')

Обращаем внимание, что помимо названий на разном языке, в них есть пробелы, которые затрудняют доступ к данным. Переименуем столбцы и проверим результат.

In [6]:
names_of = ["id","name_res","weekday","order","peop_numb"]
bo_data.set_axis(names_of,axis="columns",inplace = True) 

In [7]:
bo_data.head()

,id,name_res,weekday,order,peop_numb
0,b912c3c6a6ad,Iskra Bar,Monday,10:00,1
1,2dac0150717d,Iskra Bar,Monday,13:30,2
2,ba83aefab5dc,Iskra Bar,Tuesday,12:30,3
3,02996f1dd2ea,Iskra Bar,Wednesday,12:00,2
4,7c90be56d2ab,Iskra Bar,Thursday,14:30,3


Еще раз проверим данные на наличие пропусков, просуммировав их.

In [8]:
bo_data.isna().sum()

id            0
name_res     65
weekday       0
order         0
peop_numb     0
dtype: int64

Менеджер проекта предупреждал, что некоторые заказы заводились на тестовую карточку кафе без названия, поэтому есть смысл заполнить такой пропуск словом admin.

In [9]:
bo_data["name_res"].fillna("admin",inplace = True)

Проверяем, что пропусков не осталось.

In [10]:
bo_data.isna().sum()

id           0
name_res     0
weekday      0
order        0
peop_numb    0
dtype: int64

Установим количество явных дубликатов. Если найдутся, то удалим и снова проверим, что их не осталось.

In [11]:
bo_data.duplicated().sum()

24

In [12]:
bo_data = bo_data.drop_duplicates().reset_index(drop = True)

In [13]:
bo_data.duplicated().sum()

0

Скорее всего, такие полные дубликаты говорят о сбое при выгрузке данных. Необходимо сообщить об этом.

Еще одна проблема, о которой предупреждал менеджер проекта — дубликаты карточки Iskra Bar с неправильными названием бара - "Искра" и "Искр". На них заводились реальные заказы, поэтому необходимо вычислить их и переименовать на правильное название — Iskra Bar.

Объявим функцию `replace_restaurant()`, которая заменяет неправильное название, а потом проверяет успешность выполнения замены подсчетом (если замена прошла успешно — результат подсчета должен быть 0).

In [14]:
def replace_restaurant(df,cool,n_cool):
  return df.replace(cool,n_cool)


Передаем функции названия 'Искра' и 'Искр'.

In [15]:
bo_data = replace_restaurant(bo_data,"Искр","Iskra Bar")

In [16]:
bo_data = replace_restaurant(bo_data,"Искра","Iskra Bar")

### Выводы

* ...

## Распределение заказов в течение недели

Считаем, на какие кафе приходится больше всего заказов.

In [17]:
bo_data["name_res"].value_counts()

Iskra Bar     287
Там Ем        200
Бар Сибирь    159
Северия       107
Крафтейнер     78
Коворкафе      66
Азия           62
admin          41
Name: name_res, dtype: int64

Группируем по `weekday` и считаем, на какие дни приходится больше всего заказов.

In [18]:
bo_data["weekday"].value_counts()

Tuesday      251
Wednesday    242
Thursday     180
Monday       141
Friday       110
Saturday      52
Sunday        24
Name: weekday, dtype: int64

Объявляем функцию, которая возвращает количество заказов для ресторана в определенный день.

In [19]:
def kolvo_orders(restraunt,day):
  return bo_data[(bo_data["name_res"]== restraunt) & (bo_data["weekday"] == day)]["name_res"].count()

Передадим ей заведения Iskra Bar и Там Ем — два флагманских места, по которым просили результаты прежде всего — и понедельник, среду и пятницу в качестве дня недели.

In [20]:
monday_iskra = kolvo_orders('Iskra Bar', 'Monday')
monday_iskra

48

In [21]:
wednes_iskra = kolvo_orders("Iskra Bar","Wednesday")
wednes_iskra

53

In [22]:
frid_iskra = kolvo_orders("Iskra Bar","Friday")
frid_iskra

29

In [23]:
monday_tamem = kolvo_orders("Там Ем","Monday")
monday_tamem

25

In [24]:
wednes_tamem = kolvo_orders("Там Ем","Wednesday")
wednes_tamem


52

In [25]:
frid_tamem = kolvo_orders("Там Ем","Friday")
frid_tamem

24

Сведем результат в таблицу.

In [26]:
restraunts = ["Iskra Bar","Там Ем"]
data = [
        [f"{monday_iskra}",f"{wednes_iskra}",f"{frid_iskra}"],
        [f"{monday_tamem}",f"{wednes_tamem}",f"{frid_tamem}"]
]
columns = ["Monday","Wednesday","Friday"]
result = pd.DataFrame(index = restraunts,data = data,columns = columns)
result

,Monday,Wednesday,Friday
Iskra Bar,48,53,29
Там Ем,25,52,24


### Выводы

* ...

## Распределение заказов в течение дня

Сравним количества заказов, сделанных в обеденное и вечернее время — с 12 до 16 и с 16 до 22. Для этого объявим функцию, которая принимает данные и 2 аргумента времени.

In [69]:
bo_data['time'] = pd.to_datetime(bo_data['order']).dt.hour.apply(float)
def sravnenie(df,time_1,time_2):
  return df['name_res'][((df['time']) >= time_1) & (df['time'] < time_2)].count()


Передадим промежуток с 12:00 и 16:00

In [70]:
sravnenie_12_16 = sravnenie(bo_data,12,16)
sravnenie_12_16

543

И с 16:00 до 22:00

In [68]:
sravnenie_16_22 = sravnenie(bo_data,16,22)
sravnenie_16_22

338

### Выводы

* ...

## Типичное количество человек за столиком

Выясним, для какого количества человек обычно заказывают столик в разное время на примере Iskra Bar и Там Ем. Для этого напишем функцию, которая принимает данные, название ресторана, 2 времени и считает моду.

In [77]:
def sredn_chel(df,name,time_1,time_2):
  return df["peop_numb"][(df["time"] >= time_1)&(df["time"]<=time_2)&(df["name_res"]==name)].mode()



Проверим функцию на двух ресторанах, а также дневном и вечернем промежутке времени.

In [78]:
sredn_chel(bo_data,"Iskra Bar",12,16)

0    2
dtype: int64

In [79]:
sredn_chel(bo_data,"Там Ем",12,16)

0    2
dtype: int64

In [80]:
sredn_chel(bo_data,"Там Ем",16,22)

0    1
dtype: int64

In [81]:
sredn_chel(bo_data,"Iskra Bar",16,22)

0    1
dtype: int64

### Выводы

* ...

## Общие выводы

*ваш текст*